<a href="https://colab.research.google.com/github/Siddhantg2002/Basic-Concepts-2/blob/main/ClassificationTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEPS


1.   Import the data and required modules

 (Training/testing datasets if imported seperately then ignore step 3.)

2.   Clean the data (DATA PREPROCESSING):-
*   Seperating Input and Output (to be predicted).
*   Selecting unique values.
*   Ignoring Null Values.
*   Converting Categorical Column into Numerical using Feature Columns.
*   Using input Function to  break the data(dataframe) into epochs and into  batches to feed our model.


3.  Split the data into testing/training sets.
4.   Create a Model.

5.   Train the Model.
6.  Make Predictions.
7. Evaluate and improve untill you achieve higher accuracy.




# Classification
Now that we ve covered linear reoression it is time to talk bout classilication. Where regression was used to predict a numeric value.
classification is used to sedemte data points into classes of different labels. in this examole we will use a tensorFlow estimator to classify
Bowers
Sara wave touched on how actimatore work parties til as a bi anicker shrouch shie avample
This section is based on the following guide from the TensorFlow website. https://www.tensorflow.org/tutorials/estimator/premade

In [ ]:
!pip install -q sklearn
%tensorflow_version 2.x
from  __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc
from six.moves import urllib
from sklearn import datasets
import warnings
warnings.filterwarnings("ignore")

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
CSV_COLUMN_NAMES = ['Sepallength','Sepalwidth','Petallength', 'Petalwidth', 'Species']
SPECIES = ['Setosa','Versicolor','Virginica']
# Lets define some constants to help us later on

In [ ]:
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.cv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of Tensorflow) to grab our datasets and read them into a pandas dataframe
train.head()


,Sepallength,Sepalwidth,Petallength,Petalwidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
#SEPERATING OUTPUT
train_y=train.pop('Species')
test_y=test.pop('Species')
train.head()

,Sepallength,Sepalwidth,Petallength,Petalwidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [ ]:
train.shape

(120, 5)

In [ ]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

# Input Function
An input function basically is a function used to break the data(dataframe) into epochs and into batches to feed our model.

The TensorFlow model we are going to use requires that the data we pass it comes in as a tf. data. Dataset object. This means we must create a input function that can convert our current pandas dataframe into that object. Below you'll see a seemingly complicated input function, this is straight from the TensorFlow documentation (https://www.tensorflow.org/tutorials/estimator/linear).

I've commented as much as I can to make it understandble, but you may want to refer to the documentation for a detailed explination of each method.

In [ ]:
def input_fn(features,labels,training=True,batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    # Shuffle and repeat if you are in training mode.
    if training:
      dataset = dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

# Feature Column
In our dataset we have two different kinds of information: Categorical and Numeric Our categorical data is anything that is not numeric! For example, the sex column does not use numbers, it uses the words "male" and "female". Before we continue and create/train a model we must covet our categorical data into numeric data. We can do this by encoding each category with an integer (ex. male = 1, female = 2). Fortunately for us TensorFlow has some tools to help!

Here there is only Numeric column.

In [ ]:
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='Sepallength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Sepalwidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Petallength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Petalwidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


# Building the Model
And now we are ready to choose a model. For classification tasks there are variety of different estimators/models that we can pick from.
Some options are listed below.

• DINClassifier (Deep Neural Network)

• LinearClassifier


We can choose either model but the DNN seems to be the best choice. This is because we may not be able to find a linear corespondence in our data.

So let's build a model!

What we've just done is created a deep neural network that has two hidden layers. These layers have 30 and 10 neurons respectively. This is the number of neurons the TensorFlow official tutorial uses so we'll stick with it. However, it is worth mentioning that the number of hidden neurons is an arbitrary number and many experiments and tests are usually done to determine the best choice for these values. Try playing around with the number of hidden neurons and see if your results change.

In [ ]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
  feature_columns=my_feature_columns,
  # Two hidden layers of 30 and 10 nodes respectively.
  hidden_units=[30, 10],
  # The model must choose between 3 classes.
  n_classes=3)

# Training
The only thing to explain here is the steps argument. This simply tells the classifier to run for 5000 steps. Try modifying this and seeing if your results change. Keep in mind that more is not always better.

In [ ]:
classifier.train(
    input_fn = lambda: input_fn(train, train_y, training=True), 
    steps=5000)
    # We include a lambda to avoid creating an inner function previously

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


# Evaluation
Now let's see how this trained model does!

Notice this time we didn't specify the number of steps. This is because during evaluation the model will only look at the testing data one time.

In [ ]:
eval_result = classifier.evaluate(
  input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.900



# Predictions
Now that we have a trained model it's time to use it to make predictions. I've written a little script below that allows you to type the features of a flower and see a prediction for its class.

In [ ]:
def input_fn(features, batch_size=256):
    #Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['Sepallength','SepalWidth','Petallength','Petalwidth']
predict = {}

print("Please type numeric values as prompted.") 
for feature in features:
  valid = True
  while valid:
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}"({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))

Please type numeric values as prompted.
Sepallength: 6.4
SepalWidth: 2.8
Petallength: 5.6
Petalwidth: 2.2


ValueError: ignored